### bd econ CPS price level retrieval

bd_CPS_cpi.ipynb

November 17, 2018

@bd_econ

Requires: an API key from the [Bureau of Labor Statistics](https://www.bls.gov/developers/home.htm) (free), saved in a python file called `config.py` in the same directory as this notebook. The python file can be created with a text file and should simply contain: `bls_key = API key pasted here`.

In [1]:
# Import packages and bls api key
import pandas as pd
import numpy as np
import os
import requests
import json
import config

os.chdir('/home/brian/Documents/CPS/data/clean/')

##### BLS API retrieval

Run after BLS releases CPI data for period of interest.

In [2]:
# Code to update CPI as needed
api_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(config.bls_key)

# Series stored as a dictionary
series_dict = {'CUSR0000SA0': 'ALL',
               'CUUR0100SA0': 'Northeast',
               'CUUR0200SA0': 'Midwest',
               'CUUR0300SA0': 'South',
               'CUUR0400SA0': 'West'}

# Start year and end year
date_r = (1994, 2018)

# Handle dates
dates = [(str(date_r[0]), str(date_r[1]))]
while int(dates[-1][1]) - int(dates[-1][0]) > 10:
    dates = [(str(date_r[0]), str(date_r[0]+9))]
    d1 = int(dates[-1][0])
    while int(dates[-1][1]) < date_r[1]:
        d1 = d1 + 10
        d2 = min([date_r[1], d1+9])
        dates.append((str(d1),(d2)))
        
df = pd.DataFrame()

for start, end in dates:
    # Submit the list of series as data
    data = json.dumps({
        "seriesid": list(series_dict.keys()),
        "startyear": start, "endyear": end})

    # Post request for the data
    p = requests.post(
        '{}{}'.format(api_url, key), 
        headers={'Content-type': 'application/json'}, 
        data=data).json()
    for s in p['Results']['series']:
        col = series_dict[s['seriesID']]
        for r in s['data']:
            date = pd.to_datetime('{} {}'.format(
                r['periodName'], r['year']))
            df.set_value(date, col, float(r['value']))
df = df.sort_index()
# Output results
print('Post Request Status: {}'.format(p['status']))

cpi = (df.iloc[-1] / df)
cpi.to_csv('cpi.csv')

/home/brian/miniconda3/envs/bdcps/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Post Request Status: REQUEST_SUCCEEDED
